In [3]:
import requests
import csv
import pandas as pd


In [4]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 28.7 MB/s eta 0:00:00


In [5]:
!apt-get update

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,630 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,830 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,566 k

In [6]:
!apt-get install -y chromium-browser

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 30.2 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3build1 [159 kB]
Get:4 http://archive.ubuntu

In [7]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  chromium-chromedriver
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,308 B of archives.
After this operation, 77.8 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 chromium-chromedriver amd64 1:85.0.4183.83-0ubuntu2.22.04.1 [2,308 B]
Fetched 2,308 B in 0s (8,107 B/s)
Selecting previously unselected package chromium-chromedriver.
(Reading database ... 124092 files and directories currently installed.)
Preparing to unpack .../chromium-chromedriver_1%3a85.0.4183.83-0ubuntu2.22.04.1_amd64.deb ...
Unpacking chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...
Setting up chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# **Product Scrap**

In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


In [10]:


def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

driver.get('https://www.pickaboo.com/product-detail/creation-lamis-pure-black-edt-100-ml-for-men/')

# Scroll down to load more comments (you may need to adjust the number of times)
for _ in range(5):
    driver.find_element(By.XPATH, '//body').send_keys(Keys.PAGE_DOWN)
    time.sleep(2)  # Adjust the sleep time as needed

# Click on the "All Review" option using JavaScript
try:
  all_review_button = driver.find_element(By.XPATH, '//div[@class="all-review"]')
  driver.execute_script("arguments[0].click();", all_review_button)
  time.sleep(2)  # Allow time for reviews to load (you may need to adjust this)
except Exception as e:
  print("All Review option not found. Scraping available reviews instead.")

# Find and print all comments
comments = driver.find_elements(By.XPATH, '//div[@class="comments"]')
ratings = driver.find_elements(By.XPATH, '//div[@class="rating"]')

# Create empty lists to store data
review_data = []
rating_data = []

for comment, rating in zip(comments, ratings):
  review_text = comment.text
  rating_text = rating.text

  # Remove the "Posted on" portion from the end of the review text
  review_text = review_text.rsplit('Posted on', 1)[0].strip()

  # Remove any non-digit characters from the rating more aggressively
  rating_text = ''.join(filter(lambda x: x.isdigit() or x == '.', rating_text))

  review_data.append(review_text)
  rating_data.append(rating_text)

# Create a DataFrame
df = pd.DataFrame({'Rating': rating_data, 'Review': review_data})

# Add a "Product Name" column
df['Product Name'] = 'Creation Lamis Pure Black EDT 100 ML'


# Specify the path in your Google Drive to save the file
csv_filename = '/content/drive/MyDrive/Capstone_Data_collection/reviews_data_61.csv'

# Save the DataFrame to a CSV file in your Google Drive
df.to_csv(csv_filename, index=False)

# Display the DataFrame
print(df)

driver.quit()


All Review option not found. Scraping available reviews instead.
  Rating                                 Review  \
0    5.0  Its a very bad and horrible parcel...   

                           Product Name  
0  Creation Lamis Pure Black EDT 100 ML  
